In [1]:
# from azureml.core.environment import Environment
# Environment(name="dopplerenv")

# # Check core SDK version number
# import azureml.core

# print("SDK version:", azureml.core.VERSION)

# from azureml.core import Workspace

# ws = Workspace.from_config()

# # # Storage management
# # Create Datastore
# from azureml.core.datastore import Datastore

# doppler_blob = Datastore.register_azure_blob_container(ws, 
#                       datastore_name="doppler_scenes", 
#                       container_name="sampledata", 
#                       account_name="pipelinedata", 
#                       overwrite=True)

# def_data_store = ws.get_default_datastore()

# # Create data objects
# from azureml.core.dataset import Dataset
# from azureml.pipeline.core import PipelineData

# input_images = Dataset.File.from_files((doppler_blob, "doppler/images/"))
# label_ds = Dataset.File.from_files((doppler_blob, "doppler/labels/"))
# output_dir = PipelineData(name="scenes", datastore=def_data_store)

# # Register the dataset
# input_images = input_images.register(workspace=ws, name="input_images")
# label_ds = label_ds.register(workspace=ws, name="label_ds", create_new_version=True)

In [2]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# install('rasterio')
# install('pyshp')
# install('nexradaws')
# install('shapely')
# install('arm_pyart')
# install('netcdf4')

In [3]:
# %matplotlib inline
import tempfile
import pytz
import cartopy
import os
os.environ['PROJ_LIB'] = r'/anaconda/envs/dopplerenv/share/proj'
# os.environ['GDAL_DATA'] = r'/anaconda/envs/dopplerenv/lib/python3.8/site-packages/gdal'
# from mpl_toolkits.basemap import Basemap
import os 
import pyart
import nexradaws
import numpy as np
from osgeo import gdal
from time import time
import matplotlib.pyplot as plt
import glob
from datetime import datetime, date, timedelta
import pathlib
import rasterio as rio
import shutil
# from sklearn.preprocessing import MinMaxScaler


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/Users/brianlee/opt/anaconda3/envs/doppler/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/Users/brianlee/opt/anaconda3/envs/doppler/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _np_version_under1p16 = _nlv < LooseVersion("1.16")
/Users/brianlee/opt/anaconda3/envs/doppler/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _np_version_under1p17 = _nlv < LooseVersion("1.17")
/Users/brianlee/opt/anaconda3/envs/doppler/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:13: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _np_version_under1p18 = _nlv < LooseVersion("1.18")
/Us

In [4]:
def create_date_directories(single_date):
    
    # Create date directory
    DATEDIR = pathlib.Path(os.getcwd() + '/' + str(single_date))
    DATEDIR.mkdir(parents=True, exist_ok=True)
    RAWDIR = pathlib.Path(os.getcwd() + '/' + str(single_date) + '/1_raw/')
    RAWDIR.mkdir(parents=True, exist_ok=True)
    AGGSCANDIR = pathlib.Path(os.getcwd() + '/' + str(single_date) + '/2_scan_agg/')
    AGGSCANDIR.mkdir(parents=True, exist_ok=True)
    AGGDIR = pathlib.Path(os.getcwd() + '/' + str(single_date) + '/3_daily_agg/')
    AGGDIR.mkdir(parents=True, exist_ok=True)

    return DATEDIR, RAWDIR, AGGSCANDIR, AGGDIR
    
def return_daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def download_raw(start_date, tower, time_zone):
    
    start_date = start_date.strftime("%Y%m%d")
    start_time = '1800'
    startdatetime = "".join((start_date, start_time))
    start_date1 = datetime.strptime(startdatetime, "%Y%m%d%H%M")
    
    #download entire evening's worth of data
    end_date = start_date1 + timedelta(hours=3)
    
    conn = nexradaws.NexradAwsInterface()
    # Download data
    central_timezone = pytz.timezone(time_zone)
    radar_id = tower
    start = central_timezone.localize(start_date1)
    end = central_timezone.localize(end_date)
    scans = conn.get_avail_scans_in_range(start, end, radar_id)
    # print("There are {} scans available between {} and {}\n".format(len(scans), start, end))

    results = conn.download(scans, templocation)

    # for scan in results.iter_success():
        # print ("{} volume scan time {}".format(scan.radar_id,scan.scan_time))
    
    return results

/Users/brianlee/opt/anaconda3/envs/doppler/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
def download_reflectivity(scan, file, output_directory):
    
        ### Add reflectivity raster
        radar_reflect = scan.open_pyart()
        
        # mask out last 10 gates of each ray, this removes the "ring" around th radar.
        radar_reflect.fields['reflectivity']['data'][:, -10:] = np.ma.masked

        # exclude masked gates from the gridding
        gatefilter_reflect = pyart.filters.GateFilter(radar_reflect)
        gatefilter_reflect.exclude_transition()
        gatefilter_reflect.exclude_masked('reflectivity')
        grid_reflect = pyart.map.grid_from_radars(
            radar_reflect, 
            gatefilters=(gatefilter_reflect, ),
            grid_shape=(1, 2000, 2000),
            grid_limits=((0, 1500.0),(-70000, 70000), (-70000, 70000)),
            fields=['reflectivity'],
            gridding_algo="map_gates_to_grid",
            weighting_function='NEAREST',
            roi_func='constant_roi', constant_roi=2000.0)

        output_path = os.path.join(str(output_directory) + '/' + str(file) + '_reflectivity.tif')
        
        output_file = pyart.io.write_grid_geotiff(grid_reflect, output_path, 'reflectivity', rgb=False, warp=True, 
                                cmap='pyart_NWSRef', vmin=0, vmax=100, level=0)
        return output_file
        
def download_velocity(scan, file, output_directory):
    
        ### Add velocity raster
        radar_velocity = scan.open_pyart()
        
        # mask out last 10 gates of each ray, this removes the "ring" around th radar.
        radar_velocity.fields['velocity']['data'][:, -10:] = np.ma.masked

        # exclude masked gates from the gridding
        gatefilter_velocity = pyart.filters.GateFilter(radar_velocity)
        gatefilter_velocity.exclude_transition()
        gatefilter_velocity.exclude_masked('velocity')
        grid_velocity = pyart.map.grid_from_radars(
            radar_velocity, 
            gatefilters=(gatefilter_velocity, ),
            grid_shape=(1, 2000, 2000),
            grid_limits=((0, 1500.0),(-70000, 70000), (-70000, 70000)),
            fields=['velocity'],
            roi_func='constant_roi', constant_roi=2000.0, weighting_function='Nearest')
        
        output_path = os.path.join(str(output_directory) + '/' + str(file) + '_velocity.tif')

        output_file = pyart.io.write_grid_geotiff(grid_velocity, output_path, 'velocity', rgb=False, warp=True, 
                                cmap='pyart_NWSRef', vmin=-70, vmax=70, level=0)
        
        return output_file
    
def download_differential_phase(scan, file, output_directory):
    
         ### Add differential phase raster     
        radar_differential_phase = scan.open_pyart()
        
        # mask out last 10 gates of each ray, this removes the "ring" around th radar.
        radar_differential_phase.fields['differential_phase']['data'][:, -10:] = np.ma.masked

        # exclude masked gates from the gridding
        gatefilter_differential_phase = pyart.filters.GateFilter(radar_differential_phase)
        gatefilter_differential_phase.exclude_transition()
        gatefilter_differential_phase.exclude_masked('differential_phase')
        grid_differential_phase = pyart.map.grid_from_radars(
            radar_differential_phase, 
            gatefilters=(gatefilter_differential_phase, ),
            grid_shape=(1, 2000, 2000),
            grid_limits=((0, 1500.0),(-70000, 70000), (-70000, 70000)),
            fields=['differential_phase'],
            roi_func='constant_roi', constant_roi=2000.0, weighting_function='Nearest')
        
        output_path = os.path.join(str(output_directory) + '/' + str(file) + '_differential_phase.tif')
        
        output_file = pyart.io.write_grid_geotiff(grid_differential_phase, output_path, 'differential_phase', rgb=False, warp=True, 
                                cmap='pyart_NWSRef', vmin=-10, vmax=150, level=0)
        
        return output_file
  
def download_differential_reflectivity(scan, file, output_directory):
    
        ### Add differential reflectivity raster        
        radar_differential_reflectivity = scan.open_pyart()
        
        # mask out last 10 gates of each ray, this removes the "ring" around th radar.
        radar_differential_reflectivity.fields['differential_reflectivity']['data'][:, -10:] = np.ma.masked

        # exclude masked gates from the gridding
        gatefilter_differential_reflectivity = pyart.filters.GateFilter(radar_differential_reflectivity)
        gatefilter_differential_reflectivity.exclude_transition()
        gatefilter_differential_reflectivity.exclude_masked('differential_reflectivity')
        grid_differential_reflectivity = pyart.map.grid_from_radars(
            radar_differential_reflectivity, 
            gatefilters=(gatefilter_differential_reflectivity, ),
            grid_shape=(1, 2000, 2000),
            grid_limits=((0, 1500.0),(-70000, 70000), (-70000, 70000)),
            fields=['differential_reflectivity'],
            roi_func='constant_roi', constant_roi=2000.0, weighting_function='Nearest')
        
        output_path = os.path.join(str(output_directory) + '/' + str(file) + '_differential_reflectivity.tif')
        
        output_file = pyart.io.write_grid_geotiff(grid_differential_reflectivity, output_path, 'differential_reflectivity', rgb=False, warp=True, 
                                cmap='pyart_NWSRef', vmin=-20, vmax=20, level=0)
        
def download_cross_correlation(scan, file, output_directory):
    
        ### Add cross correlation raster
        radar_cross_correlation_ratio = scan.open_pyart()
        
        # mask out last 10 gates of each ray, this removes the "ring" around th radar.
        radar_cross_correlation_ratio.fields['cross_correlation_ratio']['data'][:, -10:] = np.ma.masked

        # exclude masked gates from the gridding
        gatefilter_cross_correlation_ratio = pyart.filters.GateFilter(radar_cross_correlation_ratio)
        gatefilter_cross_correlation_ratio.exclude_transition()
        gatefilter_cross_correlation_ratio.exclude_masked('cross_correlation_ratio')
        grid_cross_correlation_ratio = pyart.map.grid_from_radars(
            radar_cross_correlation_ratio, 
            gatefilters=(gatefilter_cross_correlation_ratio, ),
            grid_shape=(1, 2000, 2000),
            grid_limits=((0, 1500.0),(-70000, 70000), (-70000, 70000)),
            fields=['cross_correlation_ratio'],
            roi_func='constant_roi', constant_roi=2000.0, weighting_function='Nearest')
        
        output_path = os.path.join(str(output_directory) + '/' + str(file) + '_cross_correlation_ratio.tif')
        
        output_file = pyart.io.write_grid_geotiff(grid_cross_correlation_ratio, output_path, 'cross_correlation_ratio', rgb=False, warp=True, 
                                cmap='pyart_NWSRef', vmin=0, vmax=2, level=0)
        
        return output_file
    
def download_spectrum_width(scan, file, output_directory):
    
        ### Add spectrum width raster
        radar_spectrum_width = scan.open_pyart()
        
        # mask out last 10 gates of each ray, this removes the "ring" around th radar.
        radar_spectrum_width.fields['spectrum_width']['data'][:, -10:] = np.ma.masked

        # exclude masked gates from the gridding
        gatefilter_spectrum_width = pyart.filters.GateFilter(radar_spectrum_width)
        gatefilter_spectrum_width.exclude_transition()
        gatefilter_spectrum_width.exclude_masked('spectrum_width')
        grid_spectrum_width = pyart.map.grid_from_radars(
            radar_spectrum_width, 
            gatefilters=(gatefilter_spectrum_width, ),
            grid_shape=(1, 2000, 2000),
            grid_limits=((0, 15000.0),(-70000, 70000), (-70000, 70000)),
            fields=['spectrum_width'],
            roi_func='constant_roi', constant_roi=2000.0, weighting_function='Nearest')
        
        output_path = os.path.join(str(output_directory) + '/' + str(file) + '_spectrum_width.tif')
        
        output_file = pyart.io.write_grid_geotiff(grid_spectrum_width, output_path, 'spectrum_width', rgb=False, warp=True, 
                                cmap='pyart_NWSRef', vmin=0, vmax=20, level=0)
        
        return output_file

/Users/brianlee/opt/anaconda3/envs/doppler/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
os.chdir(os.path.expanduser('../data/training/sources/validation_data/tx'))
os.getcwd()

/Users/brianlee/opt/anaconda3/envs/doppler/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'/Users/brianlee/Desktop/new_bats/data/training/sources/validation_data/tx'

In [ ]:
# # Establish place
# SCRIPTSDIR = os.getcwd()

# # Change to datastorage?
# os.chdir('../data')
DATADIR = os.getcwd()

# Define start and end dates
start_date = date(2017, 8, 5)
end_date = date(2017, 8, 10)

# Define current date
current_date = datetime.today().strftime('%Y%m%d')

# Create general data directories
pathlib.Path(DATADIR + '/doppler/' + str(current_date)).mkdir(parents=True, exist_ok=True)
pathlib.Path(DATADIR + '/doppler/' + str(current_date) + '/' + str(start_date.strftime('%Y'))).mkdir(parents=True, exist_ok=True)

os.chdir(DATADIR + '/doppler/' + str(current_date) + '/' + str(start_date.strftime('%Y')))
             
YEARDIR = os.getcwd()
             
# Loop through individual dates, create directories, and apply functions.
for single_date in return_daterange(start_date, end_date):
    DATEDIR, RAWDIR, AGGSCANDIR, AGGDIR = create_date_directories(single_date.strftime("%Y%m%d"))
    # print(RAWDIR)
    
    # Make temp directory and close when done
    templocation = tempfile.mkdtemp()
    results = download_raw(single_date, 'KEWX', 'US/Pacific')
    
    for i,scan in enumerate(results.iter_success(),start=1):
        file = scan.scan_time.strftime('%Y%m%d_%H%M')
        try:
            download_reflectivity(scan, file, RAWDIR)
            download_velocity(scan, file, RAWDIR)
            download_differential_phase(scan, file, RAWDIR)
            download_differential_reflectivity(scan, file, RAWDIR)
            download_cross_correlation(scan, file, RAWDIR)
            download_spectrum_width(scan, file, RAWDIR)
            print(file)

            os.chdir(RAWDIR)

            file_list = []

            for i in os.listdir(RAWDIR):

                # match scenes from same time and combine in file.
                if os.path.isfile(os.path.join(RAWDIR,i)) and str(file) in i:
                    file_list.append(i)
            file_list.sort()
            print(file_list)


            # Read metadata of first file
            with rio.open(file_list[0]) as src0:
                meta = src0.meta

            # print('read metadata')

            # Update meta to reflect the number of layers
            meta.update(count = len(file_list))

            # print('updated metadata')
            print(file_list)

            # Read each layer and write it to stack
            with rio.open(os.path.join(str(AGGSCANDIR) + '/' + str(file) + '.tif'), 'w', **meta) as dst:
                for id, layer in enumerate(file_list, start=1):
                    # print(layer)
                    with rio.open(layer) as src1:
                        dst.write_band(id, src1.read(1))

            # file.close()

        except:
            print(file + ' did not work')
            # file.close()

    shutil.rmtree(templocation)
    
    os.chdir(YEARDIR)
    # templocation.cleanup()

/Users/brianlee/opt/anaconda3/envs/doppler/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Downloaded KEWX20170806_010105_V06
Downloaded KEWX20170806_010414_V06
Downloaded KEWX20170806_011645_V06
Downloaded KEWX20170806_010722_V06
Downloaded KEWX20170806_011339_V06
Downloaded KEWX20170806_011030_V06
Downloaded KEWX20170806_012301_V06
Downloaded KEWX20170806_011953_V06
Downloaded KEWX20170806_013601_V06
Downloaded KEWX20170806_012948_V06
Downloaded KEWX20170806_012638_V06
Downloaded KEWX20170806_013253_V06
Downloaded KEWX20170806_014240_V06
Downloaded KEWX20170806_014848_V06
Downloaded KEWX20170806_014542_V06
Downloaded KEWX20170806_013934_V06
Downloaded KEWX20170806_015151_V06
Downloaded KEWX20170806_015454_V06
Downloaded KEWX20170806_015756_V06
Downloaded KEWX20170806_020805_V06
Downloaded KEWX20170806_020059_V06
Downloaded KEWX20170806_020403_V06
Downloaded KEWX20170806_021110_V06
Downloaded KEWX20170806_022617_V06
Downloaded KEWX20170806_022226_V06
Downloaded KEWX20170806_021835_V06
Downloaded KEWX20170806_023009_V06
Downloaded KEWX20170806_021459_V06
Downloaded KEWX20170